In [1]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw
from bw2data import parameters
from sympy.physics.units import years

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
BW_PROJECT = 'lib_rm' # insert your project name here
bd.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent-3.10-cutoff' # name of ecoinvent database in your project
LIB_RM_DB = "LIB raw materials"

In [ ]:
USER_DB = ''
BG_DB = 'ecoinvent 3.8 cutoff'

In [3]:
# Custom utils defined for the parameterization
from lca_algebraic import *
from lca_algebraic.stats import * 

In [ ]:
# Recommendations from lca_algebraic

# This is better to cleanup the whole foreground model each time, and redefine it in the notebook
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
resetDb(USER_DB)

# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
resetParams()

# Defining input parameters

## Mining phases

The following parameters are included:

- ore grade,
- mining technique mix (open cast, underground, ISL),
- tailings emissions,
- mining energy mix (diesel or electricity).

### Ore grade

Ore grade decline are observed for copper, zinc, lead and nickel in Van der Voet et al (2019). 10.1111/jiec.12722, who takes information from various sources (Mudd et al. (2017), Crowson (2012), Mudd & Jowit (2013), Northey et al. (2014) and Mudd et al. (2013)). The energy ore grade relation developed above defines how energy inputs change as a function of the
ore grade. Instead of developing a function for each individual input and output, we
assume that the energy change can serve as a proxy for the change of other inputs
and outputs. Thus, we model the effects of ore grade decline on the mining process
by applying the percentage energy change to all other input and output values. The only process affected by an ore grade decline is the metal mine operation.

1 - Definition of future ore grades G(t) and G(t0) for base years
2 - Definition of energy requirements E(G) based on the energy-ore relation from literature
3 - Calculate a percentage change p(t) between the base year and the future years
4 - Calculate modelling factor, which is applied to outflows of mining processes in ecoivent to model how inputs and outputs of mining processes change

1 - We first use a distribution to generate a set of possible ore grades. This distribution reflects uncertainty in the ore grade parameters
2 - We calculate the energy requirement for each sample based on the ore grade - energy requirement for each sample

In [ ]:
# Ore grade (TRIANGULAR distribution)
ore_grade = newFloatParam(
    'ore_grade', 
    default=0.0015,
    min=    0.00001,
    max=    0.020,
    unit='dimensionless',
    distrib=DistributionType.TRIANGLE, # can be changed to LOGNORMAL or other 
    description="Uranium ore (U3O8) grade as commonly reported, from 0.02% to 20%, however we use 10 ppm – 20000 ppm (2%) as a conservative range",
    label="Uranium ore grade",
    dbname=USER_DB)

# We will be using the values from Table 4 above
# These are the official values from the IAEA's UDEPO database
og_avg = 0.001544
og_std = 0.001299

# Ore grade (LOGNORMAL distribution)
ore_grade = newFloatParam(
    'ore_grade', 
    default=og_avg,
    std=    np.log(1+(og_std/og_avg)**2)**.5, # careful, std for a lognormal distribution is actually the lognormal of the std
    min=    0.00001,
    max=    0.020,
    unit='dimensionless',
    distrib=DistributionType.LOGNORMAL,
    description="Uranium ore (U3O8) grade as commonly reported, from 0.02% to 20%, however we use 10 ppm – 20000 ppm (2%) as a conservative range",
    label="Uranium ore grade",
    dbname=USER_DB)

ore_grade.mean = np.log(og_avg/(1+(og_std/og_avg)**2)**.5)
ore_grade.sigma = og_std

### Mining techniques

In [ ]:
mining_shares = {'open_pit':.161/.935,
                'underground':.200/.935,
                'ISL':0.574/.935}

### Mining energy mix

In [ ]:
mining_energy_shares = {'diesel':0.3,
                        'electricity':0.7}

In [ ]:
# Total energy for extraction : where does that come from ???

a_op = 274
b_op = -0.482

a_ug = 317
b_ug = -0.176

a_is = 220
b_is = -0.0485

extraction_energy_open_pit = a_op * exp(b_op * ore_grade) / recovery_rate
extraction_energy_underground = a_ug * exp(b_ug * ore_grade) / recovery_rate
extraction_energy_ISL = a_is * exp(b_is * ore_grade) / recovery_rate

In [ ]:
mining_electricity_switch = newEnumParam(
    'mining_electricity_switch', 
    label='Mining electricity, grid or diesel',
    values=['dieselgenerator',
            'grid'], # You can provide a statistical weight for each value, by using a dict
    default='dieselgenerator', 
    description="Choice of electricity source for mining",
    dbname=USER_DB)

### Emissions

In [12]:
# Emissions of radon-222 from tailings
tailings_Rn222 = newFloatParam(
    'tailings_Rn222', 
    default=0.01951, min=0.01, max=1,
    distrib=DistributionType.TRIANGLE, # Distribution type, linear by default
    description="Rn222 from tailings, in Bq/s",
    label="Rn222 from tailings, in Bq/s",
    unit='Bq/s',
    dbname=USER_DB)

NameError: name 'USER_DB' is not defined

## Concentration phase 

The following parameters are included:

- xxx


In [ ]:
# Conversion phase, heat consumption
conversion_heat = newFloatParam(
    'conversion_heat', 
    default=26, min=26, max=665,
    distrib=DistributionType.TRIANGLE,
    label='Conversion heat input',
    unit='kWh/kg U in UF6',
    description="Heat required to convert 1 kg of UF6",
    dbname=USER_DB)

## Refining stage 

### List of all parameters 

In [ ]:
list_parameters() # recap of all parameters declared in the model 

In [ ]:
# And because a figure is worth 1000 words
utils._plot_params([p for p in params._param_registry().all() if p.distrib in ['linear', 'triangle', 'lognormal']], columns=5, size=2000, figsize=(16,9))
plt.savefig('output/parameters.svg')

# Transforming and updating parametrized inventories

In [ ]:
# Mining and milling
tailings         = findActivity('Tailing, from uranium milling, WNA', loc='EUR', db_name='UNEP_IRP_EUR')
open_pit         = findActivity('Uranium mine operation, open cast, WNA', loc='GLO', db_name='UNEP_IRP_EUR')
underground      = findActivity('Uranium mine operation, underground, WNA', loc='GLO', db_name='UNEP_IRP_EUR')
ISL              = findActivity('Uranium mine operation, in-situ leaching, WNA', loc='GLO', db_name='UNEP_IRP_EUR')

In [ ]:
# Copy to preserve the non-parametrized inventories 
open_pit_p = copyActivity(
    USER_DB,
    open_pit,
    'Uranium mine operation, open cast, parameterized')

In [ ]:
# We replace fixed values by parameters in the LCI 
# Mining techniques, 3 activities
open_pit_p.updateExchanges({
    # This is electricity
    'market for diesel, burned in diesel-electric generating set, 10MW*': dict(amount=mining_energy_shares['electricity'] * extraction_energy_open_pit,
                                                                              input=mining_elec_mix),
    # This is diesel used as fuel
    'market for diesel, burned in building machine*': mining_energy_shares['diesel'] * extraction_energy_open_pit
}
)

In [ ]:
# Update inventories 